In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "config"


In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

In [3]:
from etl_pipeline.config import pipeline_config
cn = pipeline_config.cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [4]:
from pipelines.ms.ms_pipeline import MSPipeline
from etl_pipeline.config import pipeline_config


/home/artur/repos/silenteight/venv2/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [6]:
# engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline.build_pipeline(MSPipeline)

The following fields have not been found in any file:
['PARTY1_POSITION_TITLE', 'AP_PARTY_CITIZENSHIP_COUNTRIES', 'AP_GOVT_IDS', 'PARTY1_ADDRESS1_CITY', 'PARTY1_NAME_ALIAS3', 'PARTY1_NAME_FULL', 'ACCOUNTNUMBER', 'AP_PARTY_RESIDENCY_COUNTRIES', 'AP_COUNTRY_OF_INCORPORATION', 'PARTY1_NAME_ALIAS4', 'AP_DOB', 'PARTY1_EMPLOYER', 'PARTY1_ID', 'PARTY1_NAME_ALIAS1', 'COUNTRY_CODE', 'ADDRESS1_STATE', 'ADDRESS1_CITY', 'AP_TAX_IDS', 'UNIQUE_KEY', 'ADDRESS1_PROVINCE', 'PARTY1_ADDRESS1_STATE', 'AP_PARTY_BIRTH_COUNTRIES', 'PARTY1_COUNTRY_CITIZENSHIP1', 'PARTY1_NAME_ALIAS2']
Field in agent configuration for: PARTY1_COUNTRY1 is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1 is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1 is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY is not registered field in pipeline.yaml
Field in agent configuration for: PARTY1_COUNTRY1 is not registered fi

## transform standardized to cleansed

In [9]:
payloads = []
for i in range(10000):
    with open(f'notebooks/sample/big_fat_flat_payload.json', 'r') as file:
        payloads.append(json.loads(file.read()))

In [10]:
import pandas as pd

data = pd.DataFrame.from_dict({"test": payloads})

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def run_pipeline(payloads):
    results = []
    for payload in payloads:
        payload_json = {key: payload[key] for key in sorted(payload)}

        payload_json = PayloadLoader().load_payload_from_json(payload_json)
        payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]

        payload = payload_json

        payload = pipeline.transform_standardized_to_cleansed(payload)
        new_payloads = pipeline.transform_cleansed_to_application(payload)
        results.append(new_payloads)
    return results

In [ ]:
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp'

In [11]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, use_memory_fs=True, nb_workers=10)

ModuleNotFoundError: No module named 'pandarallel'

In [19]:
%timeit _ = data.parallel_apply(run_pipeline, axis=1)

3min 3s ± 1.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

In [12]:
import dask.dataframe as dd

In [27]:
data_f = dd.from_pandas(data, npartitions=10)

In [40]:
def run_row(payload):
    try:
        payload_json = {key: payload[key] for key in sorted(payload)}

        payload_json = PayloadLoader().load_payload_from_json(payload_json)
        payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]

        payload = payload_json

        payload = pipeline.transform_standardized_to_cleansed(payload)
        new_payloads = pipeline.transform_cleansed_to_application(payload)
        return results
    except:
        return {}

In [41]:
def run_dataframe(df):
    df["new"] = df.test.apply(run_row) 
    return df

In [42]:
data_f.map_partitions(run_dataframe, meta=(pd.DataFrame)).compute()

TypeError: 'NoneType' object is not callable

### payload

In [9]:
from etl_pipeline.datatypes import Match

with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())
    
payload_json = {key: payload[key] for key in sorted(payload)}

payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json["match_ids"] = [
        Match(match_id=match["matchId"], match_name="dummy")
        for match in payload_json[cn.WATCHLIST_PARTY][cn.MATCH_RECORDS]
    ]
payload = payload_json

payload = pipeline.transform_standardized_to_cleansed(payload)
new_payloads = pipeline.transform_cleansed_to_application(payload)


In [10]:
len(new_payloads)

2

In [11]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '20/10/1963', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['Poland', 'PL']


In [12]:
from dataclasses import asdict
for i in new_payloads:
    i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
    i['match_ids'] = {'match_id': i['match_ids'].match_id, 'match_name': i['match_ids'].match_name}

In [23]:
def sort_list(dict_):
    if isinstance(dict_, dict):
        for key in dict_:
            dict_[key] = sort_list(dict_[key])
    if isinstance(dict_, list):
        sorted_ = [sort_list(i) for i in dict_]
        try:
            return sorted(sorted_)
        except TypeError:
            return sorted_
    return dict_

In [24]:
sort_list(new_payloads)

[{'alertedParty': {'headerInfo': {'currentVersionId': '122438658',
    'datasetId': '1044',
    'datasetName': 'R_US_Active_Address',
    'firstVersionCreatedDt': '2020-01-08T09:31:30.264-05:00',
    'inputVersionSample': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.',
    'lastVersionUpdatedDt': '2020-01-08T09:31:30.264-05:00',
    'masterId': '72951854',
    'masterVersion': '412740c151535f0756e8dbec7440b726c7a3e135',
    'stopDescriptors': {'stopDescriptor': {'name': 'Joe Doe'}},
    'uniqueCustomerId': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
   'inputRecordHist': {'inputRecords': {'createdDate': '01/08/20',
     'fields': [{'isScreenable': 'false',
       'name': 'SOURCE_REF',
       'sortOrder': '1',
       'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
      {'isScreenable': 'false',
       'name': 'UNIQUE_KEY',
       'sortOrder': '2',
       'value': 'A05003324172'},
      {'isScreenable': 'false', 'name': 'BARCODE', 'sort

In [25]:
import json

with open('tests/shared/parsed_payload.json', 'w') as f:
    json.dump(sort_list(new_payloads), f, sort_keys=True, indent=4)

In [37]:
with open(f'notebooks/sample/big_fat_flat_payload.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"]["matchRecords"]))]
payload_json["match_ids"] = [
        Match(match_id=match["matchId"], match_name="dummy")
        for match in payload_json[cn.WATCHLIST_PARTY][cn.MATCH_RECORDS]
    ]

In [38]:
from etl_pipeline.config import load_agent_configs, pipeline_config

In [39]:
payload = payload_json

## transform standardized to cleansed

In [40]:
payload = pipeline.transform_standardized_to_cleansed(payload)
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [41]:
len(new_payloads)

9

In [42]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
94590752
[] ['01/29/1971', '01', '29', '1971']
['AHMEDOUA'] ['FRANCE', 'FR']
PAYLOAD
94590752
[] ['1986', '27', '07/27/1986', '07']
['AHMEDOUA'] ['LY', 'LIBYAN ARAB JAMAHIRIYA']
PAYLOAD
94590752
[] ['1977', '07/06/1977', '07', '06']
['AHMEDOUA'] []
PAYLOAD
94590752
[] ['08', '08/17/1938', '17', '1938']
['AHMEDOUA'] ['ID', 'INDONESIA']
PAYLOAD
94590752
[] ['04/15/1958', '15', '1958', '04']
['AHMEDOUA'] ['UNITED STATES', 'US']
PAYLOAD
94590752
[] []
['AHMEDOUA'] ['RU|PR. LENINA, D. 179', 'RUSSIAN FEDERATION']
PAYLOAD
94590752
[] ['01/01/1982', '01', '1982']
['AHMEDOUA'] ['MALI|MALI', 'ML|ML']
PAYLOAD
94590752
[] ['08', '27', '08/27/1981', '1981']
['AHMEDOUA'] []
PAYLOAD
94590752
[] ['08', '08/17/1938', '17', '1938']
['AHMEDOUA'] ['ID', 'INDONESIA']


In [43]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
        
    except TypeError:
        pass
    i['match_ids'] = {'match_id': i['match_ids'].match_id, 'match_name': i['match_ids'].match_name}
with open('tests/shared/big_fat_payload_parsed.json', 'w') as f:
    json.dump(sort_list(new_payloads), f, sort_keys=True, indent=4)

In [44]:
# import pickle
# with open("tests/shared/parsed_payload.pkl", "rb") as f:
#     p = pickle.load(f)

# import pandas as pd
# reference_payload = pd.DataFrame([match for payload in p  for match in payload['watchlistParty']['matchRecords']])



In [45]:
with open(f'notebooks/sample/wm_address_in_payload_format_2_input_3_match_records.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)

payload_json["match_ids"] = [
        Match(match_id=match["matchId"], match_name="dummy")
        for match in payload_json[cn.WATCHLIST_PARTY][cn.MATCH_RECORDS]
    ]



In [46]:
payload = payload_json

In [47]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [48]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [49]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
123
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['04/31/1910', '02/31/1900'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA', 'United States'] ['United States of America', 'US|FL;US']
PAYLOAD
122438659
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['United States', 'UNITED STATES OF AMERICA'] ['Poland', 'PL']


In [50]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [51]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
        
    except TypeError:
        pass
    i['match_ids'] = {'match_id': i['match_ids'].match_id, 'match_name': i['match_ids'].match_name}
with open('tests/shared/parsed_payload_2_payload.json', 'w') as f:
    json.dump(sort_list(new_payloads), f,sort_keys=True, indent=4)

In [52]:
with open(f'notebooks/sample/wm_party_payload_without_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)

payload_json["match_ids"] = [
        Match(match_id=match["matchId"], match_name="dummy")
        for match in payload_json[cn.WATCHLIST_PARTY][cn.MATCH_RECORDS]
    ]



In [53]:
payload = payload_json

In [54]:
payload = pipeline.transform_standardized_to_cleansed(payload)

In [55]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [56]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
[] ['MAY 6, 1981']
[] ['|FL;US']
PAYLOAD
122438659
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [57]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
        
    except TypeError:
        pass
    i['match_ids'] = {'match_id': i['match_ids'].match_id, 'match_name': i['match_ids'].match_name}
with open('tests/shared/empty_payload.json', 'w') as f:
    json.dump(sort_list(new_payloads), f,sort_keys=True, indent=4)

In [58]:
with open(f'notebooks/sample/wm_party_payload_with_partial_supplemental_info.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json["match_ids"] = [
        Match(match_id=match["matchId"], match_name="dummy")
        for match in payload_json[cn.WATCHLIST_PARTY][cn.MATCH_RECORDS]
    ]




In [59]:
payload = payload_json

In [60]:
payload = pipeline.transform_standardized_to_cleansed(payload)

No parties
No accounts
No parties
No accounts
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No accounts
No parties
No accounts
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No parties
No accounts


In [61]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [37]:
from dataclasses import asdict
for i in new_payloads:
    try:
        i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"] = {j: asdict( i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"][j]) for j in sorted(i["alertedParty"]["inputRecordHist"]["inputRecords"]["INPUT_FIELD"])}
        
    except TypeError:
        pass
    i['match_ids'] = {'match_id': i['match_ids'].match_id, 'match_name': i['match_ids'].match_name}
with open('tests/shared/wm_party_payload_with_partial_supplemental_info.json', 'w') as f:
    json.dump(sort_list(new_payloads, f, sort_keys=True, indent=4)

In [88]:
for payload in new_payloads:
    print('=====================')
    print('PAYLOAD')
    print(payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
        
    match = payload['watchlistParty']['matchRecords']
    assert (match['inputVersionId'] == payload['alertedParty']['inputRecordHist']['inputRecords']['versionId'])
    print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
    try:
        print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
    except:
        break


PAYLOAD
122438658
[] ['MAY 6, 1981']
[] ['|FL;US']
PAYLOAD
122438659
[] ['MAY 6, 1981']
[] ['Poland', 'PL']


In [47]:
from etl_pipeline.config import load_agent_config

In [48]:
import pandas as pd
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

AttributeError: 'int' object has no attribute 'endswith'

In [49]:
from omegaconf import OmegaConf
alert_agents_config = {
        "alert_type": OmegaConf.load(os.path.join(CONFIG_APP_DIR, "agents", "agents.yaml"))
    }


import omegaconf
def get_fields(dict_):
    fields = []
    for key in dict_:
        
        if isinstance(dict_[key], omegaconf.dictconfig.DictConfig):
            new_fields = get_fields(dict_[key])
            fields.extend(new_fields)
        else:
#             import pdb; pdb.set_trace()
            fields.extend(dict_[key])
    return fields

fields = [i.split('.')[-1] for i in get_fields(alert_agents_config['alert_type']) if "alertedParty.inputRecordHist.inputRecords.INPUT_FIELD" not in i]
    
    

from glob import glob

files = glob("pipelines/**/*.py",  recursive=True)
files.extend(glob("etl_pipeline/**/*.py",  recursive=True))

fields_to_check = list(fields)
for file in files:
    with open(file) as f:
        text = f.read()
    new_fields_to_check = []
    
    for field in fields_to_check:
        if field in text:
            continue
        new_fields_to_check.append(field)
    fields_to_check = new_fields_to_check
        
    

fields_to_check

CONFIG_APP_DIR = os.environ["CONFIG_APP_DIR"] 

NameError: name 'CONFIG_APP_DIR' is not defined

In [57]:
import pickle

In [58]:
with open("/tmp/request.pkl", 'rb') as f:
    r = pickle.load(f)

In [ ]:
from silenteight.datasource.api.name.v1.name_pb2 import (
    NameFeatureInput,
)

In [ ]:
from google.protobuf.any_pb2 import Any

In [ ]:
some_any2 = Any()
message = NameFeatureInput()
some_any2.Pack(NameFeatureInput())
# some_any2.CopyFrom(i.feature_inputs[0].agent_feature_input)


In [ ]:
i.feature_inputs[0].agent_feature_input

In [ ]:
i.feature_inputs[0].agent_feature_input.Unpack(message)

In [ ]:
message

In [ ]:
unpacked_msg = extension_pb2.GeneralContent_Extension())


In [ ]:
NameFeatureInput.ParseFromString()

In [ ]:
from sile
i.feature_inputs[0].agent_feature_input.value.decode('utf-8')

In [ ]:
type(i.feature_inputs[0].agent_feature_input.type_url)

In [ ]:
for i in response.agent_inputs:
    print(i.feature_inputs)

In [ ]:
for 